In [1]:
import metapub
import pandas as pd
import os
import numpy as np
import re
from metapub import PubMedFetcher
from metapub import CrossRefFetcher
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
pd.options.mode.chained_assignment = None
os.chdir('/Users/diego.perez/Projects/ABIDE_Cross_Ref/')

2022-08-17 10:31:21 CMI-RSCH-MBP058 metapub.config[73534] WARNING NCBI_API_KEY was not set.
2022-08-17 10:31:23 CMI-RSCH-MBP058 elsapy.elsclient[73534] INFO Module loaded.
2022-08-17 10:31:23 CMI-RSCH-MBP058 elsapy.elsentity[73534] INFO Module loaded.
2022-08-17 10:31:23 CMI-RSCH-MBP058 elsapy.utils[73534] INFO Module loaded.
2022-08-17 10:31:23 CMI-RSCH-MBP058 elsapy.elsprofile[73534] INFO Module loaded.
2022-08-17 10:31:23 CMI-RSCH-MBP058 elsapy.elsdoc[73534] INFO Module loaded.
2022-08-17 10:31:23 CMI-RSCH-MBP058 elsapy.elssearch[73534] INFO Module loaded.


In [3]:
data = pd.read_excel('Consensus_ICC_Code.xlsx').fillna(0)
title = np.array(data['Title'])
pmids = np.array(data['Medline_PMID'])
np.nan_to_num(title)
cr = CrossRefFetcher()
PM = PubMedFetcher()

In [4]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
client = ElsClient(config['apikey'])

In [5]:
count = 0
for pmid in pmids:
    print(count)
    if pmid != 0.0:
        try:
            article = PM.article_by_pmid(pmid)
            tag = "PM"
            print('Yes PM')
        except:
            try:
                article = cr.article_by_title(title[count])
                tag = "CR"
                print('Yes CR')
            except:
                count = count + 1
                data.Notes[count] = "Failed"
                tag = "Failed"
                print('Could not find 1')
    else:
        try:
            article = cr.article_by_title(title[count])
            print('Yes CR trying PM')
            try:
                article = PM.article_by_doi(article.doi)
                tag = "PM"
                print('PM Works')
            except:
                doi_doc = FullDoc(doi=article.doi)
                doi_doc.read(client)
                if doi_doc.data is None:
                    data.Notes[count] = "Failed"
                    print('Could not find 2')
                    tag = "Failed"
                else:
                    tag = "EL"
                    print("Last resort")
        except:
            count = count + 1
            data.Notes[count] = "Failed"
            print("Could not find 3")   
    data.URL[count] = article.doi
    data.Notes[count] = article.title
    data.Tag[count] = tag
    if tag == "PM":
        if 'Review' in article.publication_types.values():
            data.Publication_Type[count] = 'Review'
        elif 'Editorial' in article.publication_types.values():
            data.Publication_Type[count] = 'Editorial'
        elif 'Case Reports' in article.publication_types.values():
            data.Publication_Type[count] = 'Case Report'
        elif 'Journal Article' in article.publication_types.values():
            data.Publication_Type[count] = 'Journal Article'
        else:
            data.Publication_Type[count] = str(article.publication_types.values())
        if article.title is None:
            data.Autism_title[count] = -99.0
            data.ASD_title[count] = -99.0
            data.Autism_Spectrum_Disorder_title[count] = -99.0
            data.ABIDE_title[count] = -99.0
            data.Autism_Brain_Imaging_Data_Exchange_title[count] = -99.0
        else:
            data.Autism_title[count] = article.title.lower().find('autism')
            data.ASD_title[count] = article.title.find('ASD')
            data.Autism_Spectrum_Disorder_title[count] = article.title.lower().find('autism spectrum disorder')
            data.ABIDE_title[count] = article.title.find('ABIDE')
            data.Autism_Brain_Imaging_Data_Exchange_title[count] = article.title.lower().find('autism brain imaging data exchange')
        if article.abstract is None:
            data.Autism_abstract[count] = -99.0
            data.ASD_abstract[count] = -99.0
            data.Autism_Spectrum_Disorder_abstract[count] = -99.0
            data.ABIDE_abstract[count] = -99.0
            data.Autism_Brain_Imaging_Data_Exchange_abstract[count] = -99.0
        else:
            data.Autism_abstract[count] = article.abstract.lower().find('autism')
            data.ASD_abstract[count] = article.abstract.find('ASD')
            data.Autism_Spectrum_Disorder_abstract[count] = article.abstract.lower().find('autism spectrum disorder')
            data.ABIDE_abstract[count] = article.abstract.find('ABIDE')
            data.Autism_Brain_Imaging_Data_Exchange_abstract[count] = article.abstract.lower().find('autism brain imaging data exchange')
        data.Medline_PMID[count] = article.pmid
    if tag == "CR":
        if str(article.reference).find('Di Martino') != -1:
            data.Citation[count] = "Yes"
        if article.title is None:
            data.Autism_title[count] = -99.0
            data.ASD_title[count] = -99.0
            data.Autism_Spectrum_Disorder_title[count] = -99.0
            data.ABIDE_title[count] = -99.0
            data.Autism_Brain_Imaging_Data_Exchange_title[count] = -99.0
        else:
            data.Autism_title[count] = str(article.title).lower().find('autism')
            data.ASD_title[count] = str(article.title).find('ASD')
            data.Autism_Spectrum_Disorder_title[count] = str(article.title).lower().find('autism spectrum disorder')
            data.ABIDE_title[count] = str(article.title).find('ABIDE')
            data.Autism_Brain_Imaging_Data_Exchange_title[count] = str(article.title).lower().find('autism brain imaging data exchange')
    elif tag == "EL":
        if doi_doc.data is None:
            count = count + 1
        else:
            data.Publication_Type[count] = doi_doc.data['coredata']['prism:aggregationType']
            data.Notes[count] = doi_doc.data['coredata']['dc:title']

            if doi_doc.data['coredata']['dc:title'] is None:
                data.Autism_title[count] = -99.0
                data.ASD_title[count] = -99.0
                data.Autism_Spectrum_Disorder_title[count] = -99.0
                data.ABIDE_title[count] = -99.0
                data.Autism_Brain_Imaging_Data_Exchange_title[count] = -99.0
            else:
                data.Autism_title[count] = doi_doc.data['coredata']['dc:title'].lower().find('autism')
                data.ASD_title[count] = doi_doc.data['coredata']['dc:title'].find('ASD')
                data.Autism_Spectrum_Disorder_title[count] = doi_doc.data['coredata']['dc:title'].lower().find('autism spectrum disorder')
                data.ABIDE_title[count] = doi_doc.data['coredata']['dc:title'].find('ABIDE')
                data.Autism_Brain_Imaging_Data_Exchange_title[count] = doi_doc.data['coredata']['dc:title'].lower().find('autism brain imaging data exchange')

            if doi_doc.data['coredata']['dc:description'] is None:
                data.Autism_abstract[count] = -99.0
                data.ASD_abstract[count] = -99.0
                data.Autism_Spectrum_Disorder_abstract[count] = -99.0
                data.ABIDE_abstract[count] = -99.0
                data.Autism_Brain_Imaging_Data_Exchange_abstract[count] = -99.0
            else:
                data.Autism_abstract[count] = doi_doc.data['coredata']['dc:description'].lower().find('autism')
                data.ASD_abstract[count] = doi_doc.data['coredata']['dc:description'].find('ASD')
                data.Autism_Spectrum_Disorder_abstract[count] = doi_doc.data['coredata']['dc:description'].lower().find('autism spectrum disorder')
                data.ABIDE_abstract[count] = doi_doc.data['coredata']['dc:description'].find('ABIDE')
                data.Autism_Brain_Imaging_Data_Exchange_abstract[count] = doi_doc.data['coredata']['dc:description'].lower().find('autism brain imaging data exchange')
    count = count + 1
    data.to_csv('Concensus_Results_8_17_22.csv')  

0
Yes CR trying PM
PM Works
1


2022-08-17 10:31:56 CMI-RSCH-MBP058 elsapy.elsclient[73534] INFO Sending GET request to https://api.elsevier.com/content/article/doi/10.3233/shti220395


Yes CR trying PM


2022-08-17 10:31:56 CMI-RSCH-MBP058 elsapy.elsentity[73534] WARNING HTTP 404 Error from https://api.elsevier.com/content/article/doi/10.3233/shti220395
and using headers {'X-ELS-APIKey': '1bb51d25ede064f38c932a4b2c6f53f5', 'User-Agent': 'elsapy-v0.5.0', 'Accept': 'application/json'}:
{"service-error":{"status":{"statusCode":"RESOURCE_NOT_FOUND","statusText":"The resource specified cannot be found."}}}


Could not find 2
2
Yes CR trying PM
PM Works
3
Yes CR trying PM
PM Works
4


...

Could not find 2
48


2022-08-17 10:33:45 CMI-RSCH-MBP058 elsapy.elsclient[73534] INFO Sending GET request to https://api.elsevier.com/content/article/doi/10.1002/aur.2058


Yes CR trying PM


2022-08-17 10:33:45 CMI-RSCH-MBP058 elsapy.elsentity[73534] WARNING HTTP 404 Error from https://api.elsevier.com/content/article/doi/10.1002/aur.2058
and using headers {'X-ELS-APIKey': '1bb51d25ede064f38c932a4b2c6f53f5', 'User-Agent': 'elsapy-v0.5.0', 'Accept': 'application/json'}:
{"service-error":{"status":{"statusCode":"RESOURCE_NOT_FOUND","statusText":"The resource specified cannot be found."}}}


Could not find 2
49
Yes CR trying PM
PM Works


In [8]:
count

2495

"Transcatheter Closure of a Secundum Atrial Septal Defect with Deficient Aortic Rim Through the Left Internal Jugular Vein in a Child with Situs Inversus and Interrupted Inferior Vena Cava: Device's Choice Matters"